In [1]:
from transformers.PreviousStateTransformer import PreviousStateTransformer
from transformers.LastStateTransformer import LastStateTransformer
from transformers.AggregateTransformer import AggregateTransformer
from transformers.IndexBasedTransformer import IndexBasedTransformer
from transformers.StaticTransformer import StaticTransformer
import pm4py
import os
from sklearn.pipeline import Pipeline


In [2]:
data = pm4py.read_xes("BPI Challenge 2017.xes.gz")

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [5]:
df = pm4py.convert_to_dataframe(data)
df.head()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pipe = Pipeline(
    steps=[
        ("AggregateTransformer", AggregateTransformer(case_id_col = 'case:concept:name',
                        cat_cols = ['Action', 'EventOrigin', 'lifecycle:transition'], 
                        num_cols = ['case:RequestedAmount' ]))
    ]
)
Aggregate_df = pipe.fit_transform(df)
Aggregate_df.head()

,Action_Created,Action_Deleted,Action_Obtained,Action_Released,Action_statechange,EventOrigin_Application,EventOrigin_Offer,EventOrigin_Workflow,lifecycle:transition_ate_abort,lifecycle:transition_complete,lifecycle:transition_resume,lifecycle:transition_schedule,lifecycle:transition_start,lifecycle:transition_suspend,lifecycle:transition_withdraw,case:RequestedAmount_mean,case:RequestedAmount_max,case:RequestedAmount_min,case:RequestedAmount_sum,case:RequestedAmount_std
case:concept:name,,,,,,,,,,,,,,,,,,,,
Application_1000086665,5,3,3,3,8,6,4,12,2,10,1,3,2,3,1,5000.0,5000.0,5000.0,110000.0,0.0
Application_1000158214,6,4,3,2,10,7,5,13,2,13,0,4,3,2,1,12500.0,12500.0,12500.0,312500.0,0.0
Application_1000311556,4,2,3,2,7,5,4,9,1,10,1,2,2,2,0,45000.0,45000.0,45000.0,810000.0,0.0
Application_1000334415,8,6,8,6,12,9,5,26,3,16,3,6,5,6,1,5000.0,5000.0,5000.0,200000.0,0.0
Application_1000339879,10,8,12,7,14,11,5,35,3,20,4,8,8,7,1,37500.0,37500.0,37500.0,1912500.0,0.0


In [5]:
pipe = Pipeline(
    steps=[
        ("IndexBasedTransformer", IndexBasedTransformer(case_id_col = 'case:concept:name',
                        cat_cols = ['Action', 'EventOrigin', 'lifecycle:transition'], 
                        num_cols = ['case:RequestedAmount' ],
                        max_events=10))
    ]
)
IndexBased_df = pipe.fit_transform(df)
Aggregate_df.head()

In [6]:
pipe = Pipeline(
    steps=[
        ("LastStateTransformer", LastStateTransformer(case_id_col = 'case:concept:name',
                        cat_cols = ['Action', 'EventOrigin', 'lifecycle:transition'], 
                        num_cols = ['case:RequestedAmount' ]
                        ))
    ]
)
LastState_df = pipe.fit_transform(df)
Aggregate_df.head()

In [7]:
pipe = Pipeline(
    steps=[
        ("PreviousStateTransformer", PreviousStateTransformer(case_id_col = 'case:concept:name',
                        cat_cols = ['Action', 'EventOrigin', 'lifecycle:transition'], 
                        num_cols = ['case:RequestedAmount' ]
                        ))
    ]
)
PreviousState_df = pipe.fit_transform(df)
Aggregate_df.head()

In [8]:
pipe = Pipeline(
    steps=[
        ("StaticTransformer", StaticTransformer(case_id_col = 'case:concept:name',
                        cat_cols = ['Action', 'EventOrigin', 'lifecycle:transition'], 
                        num_cols = ['case:RequestedAmount' ]
                        ))
    ]
)
Static_df = pipe.fit_transform(df)
Aggregate_df.head()